Importar librerias

In [8]:
import pandas as pd
import mysql.connector as connection
from google.cloud import bigquery 
import os
import pyarrow

Conexion a MySql

In [27]:
#Creacion de funcion para conectar al servidor de mysql
def conexion(host, user, password, db):
    try: 
        mydb = connection.connect(
            host= host,
            user=user,
            passwd=password,
            database=db
        )
        return mydb
    
    #Mensaje en caso no se logre la conexion al servidor   
    except connection.Error as e:
        print(f"El error en la conexion es la siguiente: {e}")
        return None


#Variables que debe ingresar el usuario para conectarse a BD
in_host = input("Favor indica el host")
in_user = input("Favor indica el user")
in_pass = input("Favor indica el tu contrasena")
in_db = input("indica la BD")

Funciones para ejecutar querys y creacion de lista con las tablas disponibles en MySql

In [28]:

#Funcion para obtener el nombre de las tablas
def ejecucion_lista(query, conec):
    try:
        cursor = conec.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        return [row[0].decode() for row in result]
    except connection.Error as e:
        print("Error al obtener el nombre de las tablas: {e}")
        return None
    
#Funcion para ejecutar un query 
def ejecucion_query(query, conec):
    try:
        df = pd.read_sql(query, conec)
        return df
    except connection.Error as e:
        print("Error en ejecutar el query: {e}")
        return None

#Definir las variables para la conexion al servidor y el query
conexion_bd = conexion(in_host, in_user, in_pass, in_db)
querry = 'SHOW TABLES'


#Si la conexion al servidor fue exitosa, ejecutar la funcion para obtener los nombres de las tablas de la BD
if conexion_bd:
    print('Conexion satisfecha')

    lista = ejecucion_lista(querry, conexion_bd)
    
    if lista is not None:
        print('Estas son las tablas disponibles en la BD:')
        print(lista)

Conexion satisfecha
Estas son las tablas disponibles en la BD:
['categories', 'customers', 'departments', 'order_items', 'orders', 'products']


Solicitud al usuario de las tablas que se requieren migrar

In [29]:
#Pedir al usuario cuale son las tablas que quiere migrar
lista_solicitada = input('Escribe una lista de tablas que quieras migrar, separado por comas: ')

#crear una lista de las tablas ingresadas por el usuario, separadas por, eliminando los espacios en blanco
lista_de_objetos = [objeto.strip() for objeto in lista_solicitada.split(',')]

#Mostrar al usuario la lista que creo
print(lista_de_objetos)


['order_items', 'tablaDummy']


Definir datos de credenciales

In [4]:
#Definir la conexion a Bigquery
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'service.json'
client = bigquery.Client()
dataset_destino = 'sqlprueba-389904.proyecto_migracion1'



Validacion de las tablas indicadas por el usuario, comparado con las tablas existentes en la BD

In [30]:
#Variable para definir las tablas que no existen en lista_de_objetos (tablas disponibles en mysql)
tablas_invalidas = [tabla for tabla in lista_de_objetos if tabla not in lista]

#Si existen tablas invalidas, genere el error, imprima las tablas incorrectas y termine el proceso
if tablas_invalidas:
    print("Error: Las siguientes tablas ingresadas no son válidas:")
    for tabla in tablas_invalidas:
        print(tabla)
    exit(1)

Error: Las siguientes tablas ingresadas no son válidas:
tablaDummy


Migracion de las tablas de Mysql a BigQuery

In [7]:
#Proceso para migrar todas las tablas definidas por el usuario
for tabla_nombre in lista_de_objetos:
    query_mysql = f"SELECT * FROM {tabla_nombre}"
    
    #Ejecuta el query en Mysql y obtiene los datos como DataFrame
    df_tabla_mysql = ejecucion_query(query_mysql, conexion_bd)
    
    #Nombre de la tabla de destino en BigQuery
    nombre_tabla_destino = f"{dataset_destino}.{tabla_nombre}"
    
    # Inserta los datos en la tabla de destino en BigQuery
    job = client.load_table_from_dataframe(df_tabla_mysql, nombre_tabla_destino)
    job.result()  # Espera a que se complete el proceso de carga
    
    print(f"Tabla '{tabla_nombre}' migrada a BigQuery como '{nombre_tabla_destino}'.")

C:\Users\luisj\AppData\Local\Temp\ipykernel_21864\1212778125.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conec)


Tabla 'categories' migrada a BigQuery como 'sqlprueba-389904.proyecto_migracion1.categories'.
